# Materials Project API Tutorial (New API)

This notebook covers the basics of using the modern Materials Project API to access materials science data.

## Setup

First, you need to obtain an API key from Materials Project:
1. Go to https://next-gen.materialsproject.org/api
2. Log in or create an account
3. Generate your API key

## Installation

```bash
pip install mp-api
```

## Documentation

- New API Documentation: https://api.materialsproject.org/docs
- Python Client Docs: https://docs.materialsproject.org/

## Video Tutorial

https://www.youtube.com/watch?v=Vuu7bNzmL8g&list=PLL0SWcFqypCl4lrzk1dMWwTUrzQZFt7y0&index=8

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mp_api.client import MPRester
from pymatgen.core import Structure, Element
import os

## Setup API Key

You can set your API key in several ways:

1. **Environment variable** (recommended):
   ```bash
   export MP_API_KEY="your_api_key_here"
   ```

2. **From a file**:
   Store your key in a text file and read it

3. **Direct input** (not recommended for shared code):
   ```python
   api_key = "your_api_key_here"
   ```

In [ ]:
# Option 1: From environment variable
api_key = os.getenv('MP_API_KEY')

# Option 2: From file
# with open('mp_api_key.txt', 'r') as f:
#     api_key = f.read().strip()

# Option 3: Direct (not recommended)
# api_key = "your_api_key_here"

if not api_key:
    print("Warning: No API key found. Please set MP_API_KEY environment variable.")

## Basic Usage: Element Properties

In [ ]:
# Basic pymatgen functionality
si = Element("Si")
print(f'Silicon has atomic mass of: {si.atomic_mass}')
print(f'Silicon atomic number: {si.Z}')
print(f'Silicon electron configuration: {si.electronic_structure}')

## Querying Materials

The new API uses a more structured approach with specific query endpoints.

In [ ]:
# Search for materials by formula
with MPRester(api_key) as mpr:
    # Search for TaC materials
    docs = mpr.materials.summary.search(formula="TaC")
    
    print(f"Found {len(docs)} TaC entries")
    
    # Display first few results
    for doc in docs[:3]:
        print(f"\nMaterial ID: {doc.material_id}")
        print(f"Formula: {doc.formula_pretty}")
        print(f"Space Group: {doc.symmetry.symbol}")
        print(f"Band Gap: {doc.band_gap} eV")
        print(f"Energy above hull: {doc.energy_above_hull} eV/atom")

## Retrieve Structure Data

In [ ]:
# Get structure for a specific material
with MPRester(api_key) as mpr:
    structure = mpr.get_structure_by_material_id("mp-1086")
    
print(structure)
print(f"\nVolume: {structure.volume:.2f} Å³")
print(f"Density: {structure.density:.2f} g/cm³")

## Advanced Searches

### Search by Chemical System

In [ ]:
# Search for materials in the Li-Fe-P-O system
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
        chemsys="Li-Fe-P-O",
        num_elements=(4, 4),  # Exactly 4 elements
        fields=["material_id", "formula_pretty", "energy_above_hull", "band_gap"]
    )
    
    # Create a DataFrame
    df = pd.DataFrame([
        {
            "material_id": doc.material_id,
            "formula": doc.formula_pretty,
            "e_above_hull": doc.energy_above_hull,
            "band_gap": doc.band_gap
        }
        for doc in docs
    ])
    
    print(f"Found {len(df)} materials in Li-Fe-P-O system")
    print(df.head(10))

### Filter by Properties

Search for stable Li-containing materials with specific band gap

In [ ]:
# Search for quaternary Li materials with specific properties
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
        elements=["Li"],  # Must contain Li
        num_elements=(4, 4),  # Quaternary
        band_gap=(0.5, 3.0),  # Band gap range
        energy_above_hull=(None, 0.01),  # Stable or near-stable
        fields=["material_id", "formula_pretty", "band_gap", "density", "energy_above_hull"]
    )
    
    df = pd.DataFrame([
        {
            "material_id": doc.material_id,
            "formula": doc.formula_pretty,
            "band_gap": doc.band_gap,
            "density": doc.density,
            "e_above_hull": doc.energy_above_hull
        }
        for doc in docs
    ])
    
    print(f"Found {len(df)} quaternary Li materials with band gap 0.5-3.0 eV")
    print(df.head(10))

## Visualize Data

In [ ]:
# Plot band gap distribution
plt.figure(figsize=(10, 6))
plt.hist(df['band_gap'].dropna(), bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Band Gap (eV)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title('Band Gap Distribution of Quaternary Li Materials', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

## Thermodynamic Data

Get formation energy and other thermodynamic properties

In [ ]:
with MPRester(api_key) as mpr:
    # Get detailed thermo data
    docs = mpr.materials.thermo.search(
        formula="LiFePO4",
        fields=["material_id", "formation_energy_per_atom", "energy_above_hull", "decomposes_to"]
    )
    
    for doc in docs:
        print(f"Material ID: {doc.material_id}")
        print(f"Formation energy: {doc.formation_energy_per_atom} eV/atom")
        print(f"Energy above hull: {doc.energy_above_hull} eV/atom")
        if doc.decomposes_to:
            print(f"Decomposes to: {doc.decomposes_to}")
        print("-" * 50)

## Electronic Structure: Band Structure and DOS

In [ ]:
# Get density of states data
with MPRester(api_key) as mpr:
    dos = mpr.get_dos_by_material_id("mp-1086")
    
if dos:
    # Plot DOS
    plt.figure(figsize=(10, 6))
    dos.get_plot()
    plt.title('Density of States for mp-1086')
    plt.show()
else:
    print("DOS data not available for this material")

## Find Materials from CIF Files

You can search for materials matching a structure from a CIF file

In [ ]:
# Load structure from CIF
cif_file = "lab6.cif"  # One of the CIF files in this folder
structure = Structure.from_file(cif_file)

print(f"Loaded structure: {structure.composition.reduced_formula}")
print(f"Space group: {structure.get_space_group_info()}")

# Search for similar structures
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
        formula=structure.composition.reduced_formula,
        fields=["material_id", "formula_pretty", "symmetry"]
    )
    
    print(f"\nFound {len(docs)} materials with formula {structure.composition.reduced_formula}")
    for doc in docs:
        print(f"  {doc.material_id}: {doc.symmetry.symbol}")

## Exercise: Explore Your Own Materials

Try these tasks:

1. Search for materials in a chemical system of your choice
2. Filter by properties (band gap, density, formation energy)
3. Create visualizations of property distributions
4. Compare structures with different space groups
5. Download and analyze DOS or band structure data

**Hint**: Use the `fields` parameter to specify exactly what data you need!

In [ ]:
# Your code here
# Example: Search for your favorite material system

with MPRester(api_key) as mpr:
    # Try your own search!
    pass

## Additional Resources

- **API Documentation**: https://api.materialsproject.org/docs
- **Python Client**: https://docs.materialsproject.org/
- **Pymatgen**: https://pymatgen.org/
- **MP Website**: https://materialsproject.org/
- **Migration Guide (Legacy to New)**: https://docs.materialsproject.org/downloading-data/differences-between-new-and-legacy-api